In [2]:
import numpy as np
import pandas as pd

In [3]:
ease = pd.read_csv('./output/1_EASE_epoch10.csv')
bert = pd.read_csv('./output/2_EASE_newversion.csv')

In [4]:
ease8 = ease.groupby('user').head(8)
ease8.head(10)

,user,item
0,11,4886
1,11,4370
2,11,8961
3,11,40815
4,11,2329
5,11,32587
6,11,37386
7,11,47
10,14,1198
11,14,2011


In [5]:
bert['prior'] = [i for i in range(8,18)]*31360

In [6]:
result = pd.concat([ease8, bert])
result

,user,item,prior
0,11,4886,NaN
1,11,4370,NaN
2,11,8961,NaN
3,11,40815,NaN
4,11,2329,NaN
...,...,...,...
313595,138493,593,13.0
313596,138493,4848,14.0
313597,138493,4022,15.0
313598,138493,53125,16.0


In [7]:
result = result.drop_duplicates(['user','item'],keep='first')
result

,user,item,prior
0,11,4886,NaN
1,11,4370,NaN
2,11,8961,NaN
3,11,40815,NaN
4,11,2329,NaN
...,...,...,...
313589,138492,1210,17.0
313592,138493,293,10.0
313596,138493,4848,14.0
313597,138493,4022,15.0


In [8]:
result = result.sort_values(['user','prior'])

In [9]:
result = result.groupby('user').head(10)
result

,user,item,prior
4,11,1704,12.0
6,11,3996,14.0
8,11,1127,16.0
0,11,4886,NaN
1,11,4370,NaN
...,...,...,...
313591,138493,2628,NaN
313592,138493,593,NaN
313593,138493,110,NaN
313594,138493,1270,NaN


In [ ]:
result[['user','item']].to_csv('./output/ratio_ensemble.csv',index=False)

In [3]:
import sys
import pandas as pd
from collections import defaultdict
from tqdm import tqdm
import argparse
import os
import glob
import pdb

In [4]:

file_list=['./output/1_EASE_epoch10.csv','./output/2_EASE_newversion.csv']




In [81]:
# Hard Voting 방식
# 특성이 다른 모델 앙상블  : seauence model + AutoEncoder model

dataframe_dict ={} 
for i in range(len(file_list)):
    file = file_list[i]
    # print(file.split("/")[-1][0])
    prior_num = int(file.split("/")[-1][0])
    
    if prior_num == 1:
        prior1_df = pd.read_csv(file)
        prior1_df = prior1_df.groupby('user').head(8)
        prior1_df['prior'] = [i for i in range(8)]*31360
        dataframe_dict["1"] = prior1_df
    else:
        prior2_df = pd.read_csv(file)
        prior2_df['prior'] = [i for i in range(8,18)]*31360
        dataframe_dict["2"] = prior2_df

result = pd.concat([dataframe_dict["1"],dataframe_dict["2"]])
result = result.drop_duplicates(['user','item'],keep='first')
result = result.sort_values(['user','prior'])
result = result.groupby('user').head(10).reset_index(drop=True)
#print(dataframe_dict)
print(result[result['user']==11])


   index  user   item  prior
0      0    11   4886      0
1      1    11   4370      1
2      2    11   8961      2
3      3    11  40815      3
4      4    11   2329      4
5      5    11  32587      5
6      6    11  37386      6
7      7    11     47      7
8      4    11   1704     12
9      6    11   3996     14


In [10]:

ratio = input(f"(모델 수 : {len(file_list)})띄어쓰기로 구분하여 ratio 입력하기")
ratio = ratio.split(" ")
ratio = list(map(int,ratio))
df_list = [pd.read_csv(f) for f in file_list]

for df,ratio in zip(df_list,ratio):
    df["weight"] = ratio
result = pd.concat(df_list).reset_index(drop=True)
g = result.groupby(['user','item'])
df =g.agg({'weight': 'sum'})
df['weight'] = - df['weight']
df = df.sort_values(['user','weight'])
df['weight'] = - df['weight']
result = result.pivot(index='user', columns='item', values='weight')  # 'user'를 기준으로 'item'과 'weight'를 컬럼으로 변환
result = df.groupby('user').head(10)
result


ValueError: Index contains duplicate entries, cannot reshape

In [15]:

ratios = input(f"(모델 수 : {len(file_list)})띄어쓰기로 구분하여 ratio 입력하기")
ratios = ratios.split(" ")
ratios = list(map(int,ratios))
if len(ratios) != len(file_list):
    raise ValueError("모델 수와 가중치값의 수가 맞지 않음") 
df_list = [pd.read_csv(f) for f in file_list]

for df,ratio in zip(df_list,ratios):
    df["weight"] = ratio
result = pd.concat(df_list).reset_index(drop=True)
g = result.groupby('user').agg({'weight': 'sum', 'item': lambda x: x[1]})

g['weight'] = - g['weight']
g = g.sort_values(['user','weight'])
g['weight'] = - g['weight']
result = g.groupby('user').head(10)
result = result.sort_values(by='user')
result

KeyError: 1